In [43]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [44]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [45]:
wine = load_wine()
wine

{'DESCR': '.. _wine_dataset:\n\nWine recognition dataset\n------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 178 (50 in each of three classes)\n    :Number of Attributes: 13 numeric, predictive attributes and the class\n    :Attribute Information:\n \t\t- Alcohol\n \t\t- Malic acid\n \t\t- Ash\n\t\t- Alcalinity of ash  \n \t\t- Magnesium\n\t\t- Total phenols\n \t\t- Flavanoids\n \t\t- Nonflavanoid phenols\n \t\t- Proanthocyanins\n\t\t- Color intensity\n \t\t- Hue\n \t\t- OD280/OD315 of diluted wines\n \t\t- Proline\n\n    - class:\n            - class_0\n            - class_1\n            - class_2\n\t\t\n    :Summary Statistics:\n    \n    ============================= ==== ===== ======= =====\n                                   Min   Max   Mean     SD\n    ============================= ==== ===== ======= =====\n    Alcohol:                      11.0  14.8    13.0   0.8\n    Malic Acid:                   0.74  5.80    2.34  1.12\n    Ash:           

In [46]:
wine_Data = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_Data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [47]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [48]:
wine_data = wine.data[:]
wine_target = wine.target[:]

In [57]:
train_x, test_x, train_y, test_y = train_test_split(wine_data, wine_target)

print(len(train_x))
print(len(test_x))

133
45


In [58]:
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).long()

test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).long()

print(train_x.shape)
print(train_y.shape)

torch.Size([133, 13])
torch.Size([133])


In [59]:
train = TensorDataset(train_x, train_y)

print(train[0])

train_loader = DataLoader(train, batch_size = 32, shuffle=True)

(tensor([1.4060e+01, 2.1500e+00, 2.6100e+00, 1.7600e+01, 1.2100e+02, 2.6000e+00,
        2.5100e+00, 3.1000e-01, 1.2500e+00, 5.0500e+00, 1.0600e+00, 3.5800e+00,
        1.2950e+03]), tensor(0))


In [70]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(13,96)
    self.fc2 = nn.Linear(96,128)
    self.fc3 = nn.Linear(128,64)
    self.fc4 = nn.Linear(64,10)
    self.fc5 = nn.Linear(10,3)

  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    x = self.fc5(x)
    return F.log_softmax(x)

model = Net()    

In [71]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(),lr = 0.001)
loss_arr = []
for epoch in range(1000):
  loss_a = 0
  for train_x, train_y in train_loader:
    train_x, train_y = Variable(train_x), Variable(train_y)

    optimizer.zero_grad()

    output = model(train_x)
    loss = criterion(output, train_y)
    loss.backward()
    optimizer.step()
  if (epoch + 1) % 50 == 0:
    print(epoch + 1, loss)    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


50 tensor(1.0592, grad_fn=<NllLossBackward>)
100 tensor(0.6881, grad_fn=<NllLossBackward>)
150 tensor(0.4328, grad_fn=<NllLossBackward>)
200 tensor(0.0516, grad_fn=<NllLossBackward>)
250 tensor(0.0015, grad_fn=<NllLossBackward>)
300 tensor(0.0279, grad_fn=<NllLossBackward>)
350 tensor(0.0451, grad_fn=<NllLossBackward>)
400 tensor(0.0957, grad_fn=<NllLossBackward>)
450 tensor(0.0254, grad_fn=<NllLossBackward>)
500 tensor(0.0006, grad_fn=<NllLossBackward>)
550 tensor(2.5034e-06, grad_fn=<NllLossBackward>)
600 tensor(0.0002, grad_fn=<NllLossBackward>)
650 tensor(0.0598, grad_fn=<NllLossBackward>)
700 tensor(0.0018, grad_fn=<NllLossBackward>)
750 tensor(2.5891e-05, grad_fn=<NllLossBackward>)
800 tensor(1.3828e-06, grad_fn=<NllLossBackward>)
850 tensor(0.0022, grad_fn=<NllLossBackward>)
900 tensor(0.0085, grad_fn=<NllLossBackward>)
950 tensor(2.1458e-07, grad_fn=<NllLossBackward>)
1000 tensor(0.0171, grad_fn=<NllLossBackward>)


In [72]:
test_x, test_y = Variable(test_x), Variable(test_y)

result = torch.max(model(test_x).data, 1)[1]

accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

accuracy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


0.9111111111111111